In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'  # default is ‘last_expr'

%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/home/mink/notebooks/CameraTraps')  # append this repo to PYTHONPATH
sys.path.append('/home/mink/lib/ai4eutils')

In [3]:
import json
import os
from collections import Counter, defaultdict
from random import sample
from shutil import copyfile
from multiprocessing.pool import ThreadPool

import numpy as np
import pandas as pd
from tqdm import tqdm

import path_utils  # ai4eutils

from data_management.megadb.schema import sequences_schema_check
from data_management.megadb.converters.cct_to_megadb import process_sequences

# fws_hawaii_kauai_forest_birds_training

In [4]:
dataset_name = 'fws_hawaii_kauai_forest_birds_training'

container_root = '/mink_disk_0/camtraps/hawaii-fws-training-drop/'  
path_prefix = '20200209-training-drop/Animals/'

path_to_output = f'/home/mink/camtraps/data/megadb_jsons/{dataset_name}.json' 
path_to_output_temp = f'/home/mink/camtraps/data/megadb_jsons/{dataset_name}_temp.json' 

## Step 0 - Add an entry to the `datasets` table

Done

## Step 1 - Prepare the `sequence` objects to insert into the database

Coarse species names in the lowest directory name, including man-made objects "Traps" and "Tunnels", which are from few locations.

No sequence or location info.

In [5]:
folder = os.path.join(container_root, path_prefix)

paths = path_utils.recursive_file_list(folder)
len(paths)
paths = sorted([p.split(folder)[1] for p in paths if path_utils.is_image_file(p)])
len(paths)

11832

11830

In [6]:
paths[-100]

'Tunnels/RCNX0364.JPG'

In [12]:
sequences = []
num_images_empty = 0

for p in paths:
    clss = p.split('/')[0].lower()
    if clss.endswith('s'):
        clss = clss[:-1]
    if clss in ['trap', 'tunnel']:
        clss = 'empty'
        num_images_empty += 1
        
    sequences.append({
        'dataset': dataset_name,
        'seq_id': p.split('.')[0].replace('/', '_').replace(' ', ''),  # just the path, replacing /
        'class': [clss],
        'images': [
            {
                'file': p,
                'frame_num': 1
            }
        ]
    })

In [15]:
num_images_empty

len(sequences)

sample(sequences, 3)

9106

11830

[{'dataset': 'fws_hawaii_kauai_forest_birds_training',
  'seq_id': 'Rats_RCNX0605',
  'class': ['rat'],
  'images': [{'file': 'Rats/RCNX0605.JPG', 'frame_num': 1}]},
 {'dataset': 'fws_hawaii_kauai_forest_birds_training',
  'seq_id': 'Traps_IMG_0251(3)',
  'class': ['empty'],
  'images': [{'file': 'Traps/IMG_0251 (3).JPG', 'frame_num': 1}]},
 {'dataset': 'fws_hawaii_kauai_forest_birds_training',
  'seq_id': 'Traps_IMG_0095(4)',
  'class': ['empty'],
  'images': [{'file': 'Traps/IMG_0095 (4).JPG', 'frame_num': 1}]}]

## Step 2 - Pass the schema check

Once your metadata are in the MegaDB format for `sequence` items, we check that they conform to the format's schema.

If the format conforms, the following messages will be printed:

```
Verified that the sequence items meet requirements not captured by the schema.
Verified that the sequence items conform to the schema.
```

For large datasets, the second step will take some time (~ a minute). 

Otherwise there will be an error message describing what's wrong. Please fix the issues until all checks are passed. You might need to write some snippets of code to loop through the `sequence` items to understand which entries have problems.

In [16]:
%%time

sequences_schema_check.sequences_schema_check(sequences)

Verified that the sequence items meet requirements not captured by the schema.
Verified that the sequence items conform to the schema.
CPU times: user 1.37 s, sys: 3.67 ms, total: 1.37 s
Wall time: 1.37 s


In [17]:
with open(path_to_output_temp, 'w', encoding='utf-8') as f:
    json.dump(sequences, f, indent=1, ensure_ascii=False)

### Step 2b - copy images to flat folder

In [18]:
def copy_file(src_path, dst_path):
    return copyfile(src_path, dst_path)

In [27]:
%%time

path_pairs = []
for seq in tqdm(sequences):
    
    if seq['class'][0] == 'empty':
        continue
    
    seq_id = seq['seq_id']
    for im in seq['images']:
        src_path = os.path.join(container_root, path_prefix, im['file'])
        assert os.path.exists(src_path), src_path
        frame = 1
        dst_path = os.path.join('/mink_disk_0/camtraps/imerit12b', 
                                f'{dataset_name}.seq{seq_id}.frame{frame}.jpg')
        path_pairs.append((src_path, dst_path))

100%|██████████| 11830/11830 [00:00<00:00, 413140.74it/s]

CPU times: user 24 ms, sys: 8.35 ms, total: 32.4 ms
Wall time: 30.8 ms


In [28]:
len(path_pairs)
path_pairs[-100]

2724

('/mink_disk_0/camtraps/hawaii-fws-training-drop/20200209-training-drop/Animals/Rats/RCNX0593.JPG',
 '/mink_disk_0/camtraps/imerit12b/fws_hawaii_kauai_forest_birds_training.seqRats_RCNX0593.frame1.jpg')

In [21]:
%%time

with ThreadPool(8) as pool:
    dst_paths = pool.starmap(copy_file, path_pairs)

CPU times: user 3.25 s, sys: 5.65 s, total: 8.9 s
Wall time: 4.22 s


In [22]:
len(dst_paths)

2724

Manually moved from `imerti12b` to `imerit12c`